# Precipitation analysis

This notebook is to analze and regress precipitaion data with results from ONI, CANI, and EANI.

# Imports

In [56]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
import cftime
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import glob
from geocat.viz import util as gvutil
import cartopy.crs as ccrs
import cartopy.feature as cf
import cartopy.util as cutil
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy.stats import t

# PBSClustter

In [57]:
# Create a PBS cluster object
cluster = PBSCluster(account='P93300313',
                     job_name='ATLN-ENSO-CESMLE2',
                     cores=1,
                     memory='8GiB',
                     processes=1,
                     walltime='01:00:00',
                     queue='casper',
                     interface='ext',
                     n_workers=1)

# dont scale many workers unless using LE
# cluster.scale(10)

client = Client(cluster)
client

/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46153 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/46153/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/46153/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.177:34943,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/acruz/proxy/46153/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [58]:
cluster.scale(2)

In [88]:
# client.shutdown()
cluster.workers

# Useful functions

In [86]:
def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit


def ds_map(ds_to_plt, bounds=[20, -60, 10, -10], name='figure'):
    fig, ax = plt.subplots(1, 1,
                           subplot_kw={'projection': ccrs.PlateCarree()})
    fig.subplots_adjust(hspace=0, wspace=0, top=0.925, left=0.1)
    cbar_ax = fig.add_axes([0, 0, 0.1, 0.1])
    cdat, clon = cutil.add_cyclic_point(ds_to_plt, ds_to_plt.longitude)

    ax.set_title(name)
    lat_ticks = np.arange(bounds[3], bounds[2], 5)
    lon_ticks = np.arange(bounds[1], bounds[0], 10)
    ax.set_xticks(lon_ticks, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_ticks, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cf.LAND)


def xr_regression(x, y, lag_x=0, lag_y=0, dim="time", alternative="two-sided"):
    """
    From https://stackoverflow.com/questions/52108417/how-to-apply-linear-regression-to-every-pixel-in-a-large-multi-dimensional-array
    requires scipy.stats as t
    Takes two xr.Datarrays of any dimensions (input data could be a 1D
    time series, or for example, have three dimensions e.g. time, lat,
    lon), and returns covariance, correlation, coefficient of
    determination, regression slope, intercept, p-value and standard
    error, and number of valid observations (n) between the two datasets
    along their aligned first dimension.

    Datasets can be provided in any order, but note that the regression
    slope and intercept will be calculated for y with respect to x.

    Inspired by:
    https://hrishichandanpurkar.blogspot.com/2017/09/vectorized-functions-for-correlation.html

    Parameters
    ----------
    x, y : xarray DataArray
        Two xarray DataArrays with any number of dimensions, both
        sharing the same first dimension
    lag_x, lag_y : int, optional
        Optional integers giving lag values to assign to either of the
        data, with lagx shifting x, and lagy shifting y with the
        specified lag amount.
    dim : str, optional
        An optional string giving the name of the dimension on which to
        align (and optionally lag) datasets. The default is 'time'.
    alternative : string, optional
        Defines the alternative hypothesis. Default is 'two-sided'.
        The following options are available:

        * 'two-sided': slope of the regression line is nonzero
        * 'less': slope of the regression line is less than zero
        * 'greater':  slope of the regression line is greater than zero

    Returns
    -------
    regression_ds : xarray.Dataset
        A dataset comparing the two input datasets along their aligned
        dimension, containing variables including covariance, correlation,
        coefficient of determination, regression slope, intercept,
        p-value and standard error, and number of valid observations (n).

    """

    # Shift x and y data if lags are specified
    if lag_x != 0:
        # If x lags y by 1, x must be shifted 1 step backwards. But as
        # the 'zero-th' value is nonexistant, xarray assigns it as
        # invalid (nan). Hence it needs to be dropped
        x = x.shift(**{dim: -lag_x}).dropna(dim=dim)

        # Next re-align the two datasets so that y adjusts to the
        # changed coordinates of x
        x, y = xr.align(x, y)

    if lag_y != 0:
        y = y.shift(**{dim: -lag_y}).dropna(dim=dim)

    # Ensure that the data are properly aligned to each other.
    x, y = xr.align(x, y)

    # Compute data length, mean and standard deviation along dim
    n = y.notnull().sum(dim=dim)
    xmean = x.mean(dim=dim)
    ymean = y.mean(dim=dim)
    xstd = x.std(dim=dim)
    ystd = y.std(dim=dim)

    # Compute covariance, correlation and coefficient of determination
    cov = ((x - xmean) * (y - ymean)).sum(dim=dim) / (n)
    cor = cov / (xstd * ystd)
    r2 = cor**2

    # Compute regression slope and intercept
    slope = cov / (xstd**2)
    intercept = ymean - xmean * slope

    # Compute t-statistics and standard error
    tstats = cor * np.sqrt(n - 2) / np.sqrt(1 - cor**2)
    stderr = slope / tstats

    # Calculate p-values for different alternative hypotheses.
    if alternative == "two-sided":
        pval = t.sf(np.abs(tstats), n - 2) * 2
    elif alternative == "greater":
        pval = t.sf(tstats, n - 2)
    elif alternative == "less":
        pval = t.cdf(np.abs(tstats), n - 2)

    # Wrap p-values into an xr.DataArray
    pval = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    # Combine into single dataset
    regression_ds = xr.merge(
        [
            cov.rename("cov").astype(np.float32),
            cor.rename("cor").astype(np.float32),
            r2.rename("r2").astype(np.float32),
            slope.rename("slope").astype(np.float32),
            intercept.rename("intercept").astype(np.float32),
            pval.rename("pvalue").astype(np.float32),
            stderr.rename("stderr").astype(np.float32),
            n.rename("n").astype(np.int16),
        ]
    )

    return regression_ds


def lag_linregress_3D(x, y, lagx=0, lagy=0):
    """
    Input: Two xr.Datarrays of any dimensions with the first dim being time. 
    Thus the input data could be a 1D time series, or for example, have three 
    dimensions (time, lat, lon).
    Datasets can be provided in any order, but note that the regression slope 
    and intercept will be calculated for y with respect to x.
    Output: Covariance, correlation, regression slope and intercept, p-value, 
    and standard error on regression between the two datasets along their 
    aligned time dimension. 
    Lag values can be assigned to either of the data, with lagx shifting x, and
    lagy shifting y, with the specified lag amount.
    https://hrishichandanpurkar.blogspot.com/2017/09/vectorized-functions-for-correlation.html
    """
    # 1. Ensure that the data are properly alinged to each other. 
    x, y = xr.align(x, y)

    # 2. Add lag information if any, and shift the data accordingly
    if lagx != 0:

        # If x lags y by 1, x must be shifted 1 step backwards. 
        # But as the 'zero-th' value is nonexistant, xr assigns it as invalid 
        # (nan). Hence it needs to be dropped
        x = x.shift(time=-lagx).dropna(dim='time')

        # Next important step is to re-align the two datasets so that y adjusts
        # to the changed coordinates of x
        x, y = xr.align(x, y)

    if lagy != 0:
        y = y.shift(time=-lagy).dropna(dim='time')
        x, y = xr.align(x, y)

    # 3. Compute data length, mean and standard deviation along time axis: 
    n = y.notnull().sum(dim='time')
    xmean = x.mean(axis=0)
    ymean = y.mean(axis=0)
    xstd = x.std(axis=0)
    ystd = y.std(axis=0)

    # 4. Compute covariance along time axis
    cov = np.sum((x - xmean) * (y - ymean), axis=0) / n

    # 5. Compute correlation along time axis
    cor = cov / (xstd * ystd)

    # 6. Compute regression slope and intercept:
    slope = cov / (xstd**2)
    intercept = ymean - xmean * slope

    # # 7. Compute P-value and standard error
    # # Compute t-statistics
    # tstats = cor * np.sqrt(n-2) / np.sqrt(1 - cor**2)
    # stderr = slope / tstats

    # pval = t.sf(tstats, n-2) * 2
    # pval = xr.DataArray(pval, dims=cor.dims, coords=cor.coords)

    return cov, cor, slope, intercept,  # pval, stderr

# Data Imports

In [61]:
files = glob.glob('/glade/campaign/collections/rda/data/d728008/gpcp_v3.2_monthly/*/*.nc4')
precip = xr.open_mfdataset(files, engine='netcdf4')
precip = precip['sat_gauge_precip']
precip.compute()

<xarray.DataArray 'sat_gauge_precip' (time: 492, lat: 360, lon: 720)> Size: 510MB
array([[[0.3376028 , 0.31622022, 0.35614043, ..., 0.35154456,
         0.40317488, 0.39641893],
        [0.33578756, 0.30677962, 0.3619819 , ..., 0.3442334 ,
         0.4184466 , 0.4141367 ],
        [0.320693  , 0.31173638, 0.372351  , ..., 0.36990678,
         0.41142496, 0.387657  ],
        ...,
        [0.10463514, 0.04420645, 0.06126066, ..., 0.3175177 ,
         0.30958372, 0.2029445 ],
        [0.09131812, 0.03955639, 0.06680226, ..., 0.31683668,
         0.30925158, 0.19534281],
        [0.09770271, 0.05784304, 0.09679842, ..., 0.31237245,
         0.29756713, 0.1884838 ]],

       [[0.39603162, 0.5406516 , 0.586973  , ..., 0.11780752,
         0.18002854, 0.31500614],
        [0.47427216, 0.64025307, 0.6513799 , ..., 0.10958626,
         0.16606998, 0.3405046 ],
        [0.44840726, 0.5706187 , 0.5835746 , ..., 0.15471238,
         0.21641284, 0.3515139 ],
...
        [0.30886766, 0.30886766, 0.30886766, ..., 0.30886766,
         0.30886766, 0.30886766],
        [0.39459535, 0.39459535, 0.39459535, ..., 0.39459535,
         0.39459535, 0.39459535],
        [0.39602637, 0.39602637, 0.39602637, ..., 0.39602637,
         0.39602637, 0.39602637]],

       [[0.29085577, 0.29085577, 0.29085577, ..., 0.29085577,
         0.29085577, 0.29085577],
        [0.30848202, 0.30848202, 0.30848202, ..., 0.30848202,
         0.30848202, 0.30848202],
        [0.31613228, 0.31613228, 0.31613228, ..., 0.31613228,
         0.31613228, 0.31613228],
        ...,
        [0.33105877, 0.33105877, 0.33105877, ..., 0.33105877,
         0.33105877, 0.33105877],
        [0.33169577, 0.33169577, 0.33169577, ..., 0.33169577,
         0.33169577, 0.33169577],
        [0.21542941, 0.21542941, 0.21542941, ..., 0.21542941,
         0.21542941, 0.21542941]]], dtype=float32)
Coordinates:
  * lon      (lon) float32 3kB -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8
  * lat      (lat) float32 1kB 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * time     (time) datetime64[ns] 4kB 1983-01-01 1983-02-01 ... 2023-12-01
Attributes:
    units:         mm/day
    long_name:     combined satellite-gauge precipitation
    valid_range:   [  0. 100.]
    cell_methods:  time: mean

# Results Import

## ONI

In [62]:
ONI = xr.open_dataset('Results/ONI.nc')
ONI

<xarray.Dataset> Size: 30kB
Dimensions:  (time: 1260)
Coordinates:
  * time     (time) datetime64[ns] 10kB 1920-01-16T12:00:00 ... 2024-12-16T12...
    month    (time) int64 10kB ...
Data variables:
    ONI      (time) float64 10kB ...

## CANI and EANI

In [78]:
AN_hadisst = xr.open_dataset('Results/CANI_EANI_HADISST.nc', engine='netcdf4')
CANI = AN_hadisst['CANI']
EANI = AN_hadisst['EANI']

In [79]:
AN_hadisst

<xarray.Dataset> Size: 60kB
Dimensions:  (time: 1260)
Coordinates:
  * time     (time) datetime64[ns] 10kB 1920-01-01 1920-02-01 ... 2024-12-01
Data variables:
    EANI     (time) float64 10kB ...
    CANI     (time) float64 10kB ...
    PC1      (time) float64 10kB ...
    PC2      (time) float64 10kB ...
    PC3      (time) float64 10kB ...
Attributes:
    title:        Atlantic Niño Index Timeseries
    description:  Contains EANI, CANI, and PCs from EOFa done on HADISSTv1.1
    created:      2025-06-11

# select data

In [80]:
# selected dates
dates = xr.date_range(start='1983-01-01', end='2023-12-01', freq='1ME')
CANIs = CANI.sel(time=dates, method='nearest')
EANIs = EANI.sel(time=dates, method='nearest')

In [81]:
# select onset for convection insight
mam_precip = precip.where(precip['time.season'] == 'MAM', drop=True).compute()
mam_CANIs = CANIs.where(CANIs['time.season'] == 'MAM', drop=True).compute()
mam_EANIs = EANIs.where(EANIs['time.season'] == 'MAM', drop=True).compute()
# select summer
jja_precip = precip.where(precip['time.season'] == 'JJA', drop=True).compute()
jja_CANIs = CANIs.where(CANIs['time.season'] == 'JJA', drop=True).compute()
jja_EANIs = EANIs.where(EANIs['time.season'] == 'JJA', drop=True).compute()
# select after peak for lasting conditions
son_precip = precip.where(precip['time.season'] == 'SON', drop=True).compute()
son_CANIs = CANIs.where(CANIs['time.season'] == 'SON', drop=True).compute()
son_EANIs = EANIs.where(EANIs['time.season'] == 'SON', drop=True).compute()

In [82]:
mam_CANIs

<xarray.DataArray 'CANI' (time: 123)> Size: 984B
array([-0.74038882, -0.90929986, -1.19340762,  0.52408858,  0.46131992,
        0.42231394,  0.1543612 , -0.09168697, -0.1219812 , -0.22942006,
        0.04243752,  0.01899788, -0.44376791, -0.06254611,  0.45771916,
        1.3345065 ,  1.56288074,  1.63155045, -0.03204785,  0.04112656,
        0.21253966, -0.72754649, -0.53050983, -0.4151525 , -0.05787639,
        0.10293979,  0.11881351, -1.92144874, -2.21304691, -2.49521611,
       -1.32242399, -1.0186811 , -0.92642874,  0.11448872, -0.24947502,
       -0.52947622,  0.69494503,  0.95385874,  1.02475739,  0.47636242,
        0.99514633,  1.3497453 , -1.49573433, -1.79212421, -1.94877132,
        1.53367745,  1.38819357,  1.19721289,  0.02452396,  0.21946209,
        0.62950934,  0.36896851,  0.30753557,  0.08068861, -0.33499262,
       -0.12702372,  0.00521491, -0.22626729, -0.21744763, -0.06745711,
        0.26775427,  0.01685623, -0.01352875,  0.03014727, -0.43141438,
       -0.64626442,  0.35769249, -0.02851211, -0.40078618, -0.06763234,
        0.13021759,  0.22482212, -0.18326322, -0.03710357, -0.11520068,
        0.70663978,  1.02937351,  1.26390099,  0.24400034,  0.41932188,
        0.45578177,  1.4701587 ,  1.84139349,  2.06666238,  0.8104145 ,
        0.6276882 ,  0.5241777 , -1.93705429, -1.84627947, -1.62142385,
        0.28912588,  0.21609895,  0.10210903, -0.64923671, -0.38244523,
       -0.31347726, -0.40837823, -0.27398802, -0.30996418,  0.46092129,
        0.37204525,  0.45775872,  0.34295624,  0.35962619,  0.54539904,
       -0.2453399 , -0.48384813, -0.33050387,  0.86863446,  0.72414166,
        0.54590768,  2.06022114,  1.66690816,  1.23795587, -0.19663034,
        0.09923222,  0.73590669,  0.50800876,  0.5640607 ,  0.40328215,
        0.86677739,  0.76010809,  0.83007625])
Coordinates:
  * time     (time) datetime64[ns] 984B 1983-03-01 1983-04-01 ... 2023-05-01
Attributes:
    description:  Central Atlantic Niño Index. PCs scaled by STD and roll mea...

# regression

select per season

In [87]:
# CANIvprecip_mam = lag_linregress_3D(mam_CANIs, mam_precip)
CANIvprecip_mam = xr_regression(mam_CANIs, mam_precip).compute()
CANIvprecip_mam

<xarray.Dataset> Size: 8MB
Dimensions:    (lon: 720, lat: 360)
Coordinates:
  * lon        (lon) float32 3kB -179.8 -179.2 -178.8 ... 178.8 179.2 179.8
  * lat        (lat) float32 1kB 89.75 89.25 88.75 ... -88.75 -89.25 -89.75
Data variables:
    cov        (lat, lon) float32 1MB -0.05473 -0.02893 ... -0.09608 -0.1069
    cor        (lat, lon) float32 1MB -0.1125 -0.06933 ... -0.1139 -0.1188
    r2         (lat, lon) float32 1MB 0.01265 0.004807 ... 0.01298 0.01412
    slope      (lat, lon) float32 1MB -0.07272 -0.03844 ... -0.1277 -0.142
    intercept  (lat, lon) float32 1MB 0.4106 0.391 0.3287 ... 0.8757 0.9206
    pvalue     (lat, lon) float32 1MB 0.2155 0.4461 0.5758 ... 0.2096 0.1905
    stderr     (lat, lon) float32 1MB 0.05841 0.05028 0.03412 ... 0.1012 0.1079
    n          (lat, lon) int16 518kB 123 123 123 123 123 ... 123 123 123 123